# Local Agent Evaluation Framework


This goal of this framework is to allow for easier evaluation of agents performance in different scenarios, enabling quicker feedback to iterate through your different strategies. 

The framework provides reusable functions, and the flexibility to change different parameter inputs, scenarios, and configurations.

Gives the ability to test how well your agent's strategies are working in different scenarios against different agents. Tets to see if their is improvement in scenario performances as you update your agents, and identify any regression across different scenarios as well


* [Install Setup](#install-setup)
* [Imports](#imports)
* [Viz Toll Setup](#viz-setup)
* [Agents for Evaluation](#agents)
* [Pool Play + Scoreboard Implementation](#pplay-scoreboard)
* [Example 1 - Academy 3 vs 1 with Keeper](#example-1)
    * [Pool Play Run](#pool-play-run-ex1)
    * [Visualize Scenario Gameplay](#viz-gameplay-ex1)
* [Example 2 - Kaggle 11 vs 11 Scenario](#example-2)
* [Example 3 - Writing to files flag](#example-3)

Credit to the following notebooks for open agents I used here for evaluation and visualizations for testing:
* https://www.kaggle.com/kwabenantim/gfootball-academy
* https://www.kaggle.com/eugenkeil/simple-baseline-bot
* https://www.kaggle.com/yegorbiryukov/gfootball-with-memory-patterns
* https://www.kaggle.com/mlconsult/best-open-rules-bot-score-1020-7
* https://www.kaggle.com/jaronmichal/human-readable-visualization

<a id="install-setup"></a>
# Install Setup

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.6 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

<a id="imports"></a>
# Imports

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import time

from kaggle_environments import make
from kaggle_environments.envs.football.helpers import *
from IPython.display import HTML

pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.options.display.max_rows = 150
pd.set_option('display.max_columns', 150)
np.set_printoptions(suppress=True)

<a id="agents"></a>
# Agents for evaluation

I included agents already written to python files which come from agents open sourced by others. They can be found in directory:  
**../input/open-gfball-agents/** 

In [ ]:
!ls ../input/open-gfball-agents/

Here is writing another new agent implementation to a submission.py file. This included since a lot of other reference notebooks show the agent implementation when they write to submission.py within their kaggle notebooks. 

This is the basic template bot from here: 
https://www.kaggle.com/piotrstanczyk/gfootball-template-bot/data

In [ ]:
%%writefile template_bot_submission.py
from kaggle_environments.envs.football.helpers import *

@human_readable_agent
def agent(obs):
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return Action.Shot
        # Run towards the goal otherwise.
        return Action.Right
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

In [ ]:
!ls

<a id="viz-setup"></a>
# Viz. Tool Setup

setup visualization tool script taken from [Human Readable Visualization](https://www.kaggle.com/jaronmichal/human-readable-visualization).

In [ ]:
%%writefile visualizer.py

from matplotlib import animation, patches, rcParams
from matplotlib import pyplot as plt
from kaggle_environments.envs.football.helpers import *

WIDTH = 110
HEIGHT = 46.2
PADDING = 10


def initFigure(figwidth=12):
    figheight = figwidth * (HEIGHT + 2 * PADDING) / (WIDTH + 2 * PADDING)

    fig = plt.figure(figsize=(figwidth, figheight))
    ax = plt.axes(xlim=(-PADDING, WIDTH + PADDING), ylim=(-PADDING, HEIGHT + PADDING))
    plt.axis("off")
    return fig, ax


def drawPitch(ax):
    paint = "white"

    # Grass around pitch
    rect = patches.Rectangle((-PADDING / 2, -PADDING / 2), WIDTH + PADDING, HEIGHT + PADDING,
                             lw=1, ec="black", fc="#3f995b", capstyle="round")
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), WIDTH, HEIGHT, ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([WIDTH / 2, WIDTH / 2], [0, HEIGHT], color=paint, lw=2)

    # Dots
    dots_x = [11, WIDTH / 2, WIDTH - 11]
    for x in dots_x:
        plt.plot(x, HEIGHT / 2, "o", color=paint, lw=2)

    # Penalty box
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (HEIGHT - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y),
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, penalty_box_pos_y), -
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y),
                         goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_box_pos_y),
                         -goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goals
    goal_width = 0.044 / 0.42 * HEIGHT
    goal_pos_y = (HEIGHT / 2 - goal_width / 2)
    rect = plt.Rectangle((0, goal_pos_y), -2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_pos_y), 2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([WIDTH / 2, HEIGHT / 2], 9.15, color=paint, fc="None", lw=2)
    ax.add_artist(mid_circle)

    # Penalty box arcs
    left = patches.Arc([11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                       color=paint, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([WIDTH - 11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                        color=paint, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [WIDTH, 0], [WIDTH, HEIGHT], [0, HEIGHT]]
    angle = 0
    for x, y in corners:
        c = patches.Arc([x, y], 2, 2,
                        color=paint, fc="None", lw=2, angle=angle, theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90


def scale_x(x):
    return (x + 1) * (WIDTH / 2)


def scale_y(y):
    return (y + 0.42) * (HEIGHT / 0.42 / 2)


def extract_data(raw_obs):
    obs = raw_obs[0]["observation"]["players_raw"][0]
    res = dict()
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x), scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]

    left_active = raw_obs[0]["observation"]["players_raw"][0]["active"]
    res["left_player"] = res["left_team"][left_active]

    right_active = raw_obs[1]["observation"]["players_raw"][0]["active"]
    res["right_player"] = res["right_team"][right_active]

    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res


def draw_team(obs, team, side):
    x_coords, y_coords = zip(*obs[side])
    team.set_data(x_coords, y_coords)


def draw_ball(obs, ball):
    ball.set_markersize(8 + obs["ball"][2])  # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])


def draw_active_players(obs, left_player, right_player):
    x1, y1 = obs["left_player"]
    left_player.set_data(x1, y1)

    x2, y2 = obs["right_player"]
    right_player.set_data(x2, y2)

    if obs["ball_owned_team"] == 0:
        left_player.set_markerfacecolor("yellow")
        left_player.set_markersize(20)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)
    elif obs["ball_owned_team"] == 1:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("yellow")
        right_player.set_markersize(20)
    else:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)


def draw_team_active(obs, team_left_active, team_right_active):
    team_left_active.set_data(WIDTH / 2 - 7, -7)
    team_right_active.set_data(WIDTH / 2 + 7, -7)

    if obs["ball_owned_team"] == 0:
        team_left_active.set_markerfacecolor("indianred")
    else:
        team_left_active.set_markerfacecolor("mistyrose")

    if obs["ball_owned_team"] == 1:
        team_right_active.set_markerfacecolor("royalblue")
    else:
        team_right_active.set_markerfacecolor("lightcyan")


def draw_players_directions(obs, directions, side):
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = (x_dir ** 2 + y_dir ** 2)**0.5 + 0.00001  # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1]
        directions[i + index].set_data([x, x + x_dir / dist], [y, y + y_dir / dist])


def player_actions(step, side):
    if side == 0:
        actions = {0: "idle", 1: "←", 2: "↖", 3: "↑", 4: "↗", 5: "→", 6: "↘", 7: "↓", 8: "↙",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "stp_drb"}
    else:
        actions = {0: "idle", 1: "→", 2: "↘", 3: "↓", 4: "↙", 5: "←", 6: "↖", 7: "↑", 8: "↗",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "stp_drb"}

    obs = step[side]["observation"]["players_raw"][0]

    if obs["sticky_actions"][8]:
        spr = "+spr"
    else:
        spr = "-spr"

    if obs["sticky_actions"][9]:
        drb = "+drb"
    else:
        drb = "-drb"

    if 1 in obs["sticky_actions"][0:8]:
        i = obs["sticky_actions"][0:8].index(1) + 1
        drn = actions[i]
    else:
        drn = "|"

    if step[side]["action"]:
        act = actions[step[side]["action"][0]]
    else:
        act = "idle"

    return f"{spr} {drb} {drn} [{act}]".ljust(24, " ")


steps = None
drawings = None
directions = None
ball = left_player = right_player = None
team_left = team_right = None
team_left_active = team_right_active = None
team_left_actions = team_right_actions = None
team_left_number = team_right_number = None
team_left_direction = team_right_direction = None
text_frame = game_mode = match_info = None


def init():
    ball.set_data([], [])
    left_player.set_data([], [])
    right_player.set_data([], [])
    team_left.set_data([], [])
    team_right.set_data([], [])
    team_left_active.set_data([], [])
    team_right_active.set_data([], [])
    return drawings


def animate(i):
    obs = extract_data(steps[i])

    # Draw info about ball possesion
    draw_active_players(obs, left_player, right_player)
    draw_team_active(obs, team_left_active, team_right_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Step {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"{obs['game_mode']} Mode")

    score_a, score_b = obs["score"]
    match_info.set_text(f"{score_a} : {score_b}")

    team_left_actions.set_text(player_actions(steps[i], 0))
    team_right_actions.set_text(player_actions(steps[i], 1))

    team_left_number.set_text(str(steps[i][0]["observation"]["players_raw"][0]["active"]))
    team_right_number.set_text(str(steps[i][1]["observation"]["players_raw"][0]["active"]))

    return drawings


def visualize(trace):
    global steps
    global drawings
    global directions
    global ball, left_player, right_player
    global team_left, team_right
    global team_left_active, team_right_active
    global text_frame, game_mode, match_info
    global team_left_actions, team_right_actions
    global team_left_number, team_right_number
    global team_left_direction, team_right_direction

    rcParams['font.family'] = 'monospace'
    rcParams['font.size'] = 12

    steps = trace

    fig, ax = initFigure()
    drawPitch(ax)
    ax.invert_yaxis()

    left_player, = ax.plot([], [], "o", ms=18, mfc="firebrick", mew=0, alpha=0.5)
    right_player, = ax.plot([], [], "o", ms=18, mfc="blue", mew=0, alpha=0.5)
    team_left, = ax.plot([], [], "o", ms=12, mfc="firebrick", mew=1, mec="white")
    team_right, = ax.plot([], [], "o", ms=12, mfc="blue", mew=1, mec="white")
    ball, = ax.plot([], [], "o", ms=8, mfc="wheat", mew=1, mec="black")

    team_left_active, = ax.plot([], [], "o", ms=16, mfc="mistyrose", mec="None")
    team_right_active, = ax.plot([], [], "o", ms=16, mfc="lightcyan", mec="None")

    textheight = -6
    text_frame = ax.text(-5, textheight, "", ha="left")
    match_info = ax.text(WIDTH / 2, textheight, "", ha="center", fontweight="bold")
    game_mode = ax.text(WIDTH + 5, textheight, "", ha="right")

    team_left_actions = ax.text(WIDTH / 4 + 2, textheight, "", ha="center")
    team_right_actions = ax.text(3 * WIDTH / 4 + 2, textheight, "", ha="center")

    team_left_number = ax.text(WIDTH / 2 - 7, -6.3, "", ha="center", fontsize=10)
    team_right_number = ax.text(WIDTH / 2 + 7, -6.3, "", ha="center", fontsize=10)

    # Drawing of directions definitely can be done in a better way
    directions = []
    for _ in range(22):
        direction, = ax.plot([], [], color="yellow", lw=1.5)
        directions.append(direction)

    drawings = [team_left_active, team_right_active, left_player, right_player,
                team_left, team_right, ball, text_frame, match_info,
                game_mode, team_left_actions, team_right_actions, team_left_number, team_right_number]

    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True,
                                   interval=100, frames=len(steps), repeat=True)
    return anim


<a id="pplay-scoreboard"></a>
# Pool + and Scoreboard Implementation

In [ ]:
def run_pool_play(scenario_name, episode_steps, num_rounds, agents, export_dir, write_file=False):
    print('run pool play scenario: {0}'.format(scenario_name))
    start_time = time.time()
    
    agents_dirs = agents
    
    env_config = {
        "save_video": False,
        "scenario_name": scenario_name,
        "running_in_notebook": True,
        "episodeSteps": episode_steps
    }
    
    env = make("football", configuration=env_config, debug=False)
    
    
    df_list = []

    for pool_play_round in range(num_rounds):
        for agent1 in agents_dirs:
            for agent2 in agents_dirs:
                print(agent1, agent2)
                env.reset()
                output = env.run([agent1, agent2])

                final_output = output[-1]
                left_agent_foutput = final_output[0]
                right_agent_foutput = final_output[1]
                left_reward = left_agent_foutput['reward']
                right_reward = right_agent_foutput['reward']
                left_status = left_agent_foutput['status']
                right_status = right_agent_foutput['status']

                left_score = output[-1][0]['observation']['players_raw'][0]['score'][0]
                right_score = output[-1][0]['observation']['players_raw'][0]['score'][1]

                adf = pd.DataFrame()
                adf['scenario'] = [scenario_name]
                adf['round'] = [pool_play_round]
                adf['left_agent'] = [agent1]
                adf['right_agent'] = [agent2]
                adf['left_score'] = [left_score]
                adf['right_score'] = [right_score]
                adf['left_reward'] = [left_reward]
                adf['right_reward'] = [right_reward]
                adf['left_status'] = [left_status]
                adf['right_status'] = [right_status]

                df_list.append(adf)

        pool_play_round +=1
        
    #final dataframe with results    
    fdf = pd.concat(df_list)
    
    # write out results to file if flag set
    if write_file:
        # if export_dir does not exist then create it
        if export_dir not in os.listdir('.'):
            print('{0} directory does not exist, creating'.format(export_dir) )
            os.mkdir(export_dir)
        
        # make export directory w/ timestamp of runs
        curr_datetime = dt.datetime.now()
        curr_time = curr_datetime.strftime('%m-%d-%Y-%H-%M-%S')
        export_fdir = export_dir + curr_time
        os.mkdir(export_fdir)

        # write out results
        export_result_file = export_fdir + '/results.csv'
        fdf.to_csv(export_result_file, index=False)
        print('results written out to {0}'.format(export_result_file))

        # write out config
        config_df = pd.DataFrame(env_config.items())
        config_df = config_df.append([['num_rounds', num_rounds]])
        config_df.to_csv(export_fdir + '/config.csv', index=False)

        end_time = round((time.time() - start_time), 2)
        print("complete: --- %s seconds ---" % end_time)
    
    # get scoreboard result dataframe
    score_df = get_scoreboard(fdf)
    # add some final columns
    score_df['scenario_name'] = scenario_name
    score_df = score_df[['scenario_name','agent', 'games_played', 'num_wins', 'num_losses', 'num_ties',
       'goals_for', 'goals_against', 'num_points']]
    
    return fdf,score_df


def get_scoreboard_from_file(result_file):
    rdf = pd.read_csv(result_file)
    score_df = get_scoreboard(rdf)
    return score_df


def get_scoreboard(rdf):
    # get only cases where valid statuses
    rdf = rdf[rdf['right_status'] == 'DONE']
    rdf = rdf[rdf['left_status'] == 'DONE']

    agents = list(set(list(rdf.left_agent.unique()) + list(rdf.right_agent.unique())))
    
    result_list = []

    for agent in agents:
        left_df = rdf[rdf.left_agent == agent].reset_index(drop=True)

        # calculate num_wins, num_losses, num_ties
        left_df['num_wins'] = np.where(left_df['left_score'] > left_df['right_score'], 1, 0)
        left_df['num_losses'] = np.where(left_df['left_score'] < left_df['right_score'], 1, 0)
        left_df['num_ties'] = np.where(left_df['left_score'] == left_df['right_score'], 1, 0)

        games_played = len(left_df)
        goals_for = left_df.left_score.sum()
        goals_against = left_df.right_score.sum()
        num_wins = left_df.num_wins.sum()
        num_losses = left_df.num_losses.sum()
        num_ties = left_df.num_ties.sum()

        right_df = rdf[rdf.right_agent == agent].reset_index(drop=True)

        # calculate num_wins, num_losses, num_ties
        right_df['num_wins'] = np.where(right_df['right_score'] > right_df['left_score'], 1, 0)
        right_df['num_losses'] = np.where(right_df['right_score'] < right_df['left_score'], 1, 0)
        right_df['num_ties'] = np.where(right_df['right_score'] == right_df['left_score'], 1, 0)

        games_played = games_played + len(right_df)
        goals_for = goals_for + right_df.right_score.sum()
        goals_against = goals_against + right_df.left_score.sum()
        num_wins = num_wins + right_df.num_wins.sum()
        num_losses = num_losses + right_df.num_losses.sum()
        num_ties = num_ties + right_df.num_ties.sum()

        result_list.append([agent, games_played, num_wins, num_losses, num_ties, goals_for, goals_against])

    fdf = pd.DataFrame(result_list, columns = ['agent', 'games_played', 'num_wins', 'num_losses', 'num_ties',
                                               'goals_for', 'goals_against'])

    fdf['num_points'] = fdf['num_wins']*3 + fdf['num_ties']*1

    fdf = fdf.sort_values('num_points', ascending=False)
    
    return fdf

<a id="example-1"></a>
# Example 1 - Academy 3 vs 1 with Keeper

<a id="pool-play-run-ex1"></a>
### Pool Play Run

In [ ]:
#AGENTS_DIR = '../input/open-gfball-agents/' # where the agents submission.py files are located
EXPORT_DIR = 'pool_play_results/' # optional flag to store results of pool play to files
NUM_ROUNDS = 3 # number of time to repeat playing every agent in pool

AGENTS = [
    '../input/open-gfball-agents/best_open_rules_bot_1020_v2.py',
    '../input/open-gfball-agents/tunable_baseline_bot.py',
    '../input/open-gfball-agents/gfball_pattern_v15_submission.py',
    'template_bot_submission.py'
]

EPISODE_STEPS = 400 # lets do shorter than default 3000 to speed up runs

SCENARIO_NAME = 'academy_3_vs_1_with_keeper'

In [ ]:
result_df, score_df = run_pool_play(SCENARIO_NAME, EPISODE_STEPS, NUM_ROUNDS, AGENTS, 
                                    EXPORT_DIR, write_file=False)

The score_df has the aggregated results of the different scenario runs of agents against each other.

Since this is a academy scenario the way you interpet the metrics are not as intuitive vs the typical 11v11_kaggle scenario (which you can see in the next section)

For example lets take agent **gfball_pattern_v15_submission** below:
* 18 total games played is because the number of rounds = 3 for the input and there are 3 agents played against including itself. 3x3 = 9 for being on offense for the scenario, and 9 for being on defense for the scenario.
* From the goals for you can see that 3/9 matches on offense the agent was able to score, and 2/9 times on defense agent let in a goal.  so 6/9 times on offense wasnt able to score, and 7/9 times on defense didnt allow a goal. 6+7 = 13 ties.

In [ ]:
score_df

If you want to Look at the results of each match, see result_df

In [ ]:
# 48 total matches played, 4 teams playing each other including itself x 4 teams total = 16 x 3 rounds = 48
print(result_df.shape) 
result_df.head(20)

<a id="viz-gameplay-ex1"></a>
### Visualize Scenario Gameplay

So lets say our agent is tunable_baseline_bot, and we want to understand why the agents performance is not great for this academy 3 vs 1 scenario, lets run and visualize a bit

In [ ]:
def agent_run(left_agent, right_agent, debug, config):
    env = make("football", debug=debug, configuration=config)
         
    output = env.run([left_agent, right_agent])
    print('SCENARIO TYPE: {0}'.format(config['scenario_name']))
    print(left_agent + ' vs '+ right_agent)
    scores = output[-1][0][
        "observation"]["players_raw"][0]["score"]
    print("Scores  {0} : {1}".format(*scores))
    print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))
    print('number of steps: {0}'.format(len(output)))
    return output

In [ ]:
CONFIG={
    "save_video": False, 
    "scenario_name": "academy_3_vs_1_with_keeper", 
    "running_in_notebook": True
}

DEBUG = False

LEFT_AGENT = "../input/open-gfball-agents/tunable_baseline_bot.py"
RIGHT_AGENT = "../input/open-gfball-agents/best_open_rules_bot_1020_v2.py"

In [ ]:
%%time
output = agent_run(LEFT_AGENT, RIGHT_AGENT, DEBUG, CONFIG)

In [ ]:
from visualizer import visualize
viz = visualize(output)
HTML(viz.to_html5_video())

We can see above that tunable-baseline-bot needs to improve offense strategy to score in this type of scenario,

so just make changes to your agent, and now you can rerun evaluations with the methods as described above to check if your agent is improveing!

Hmm, what about the gfootball agent offense vs tunable baseline defense for this academy 3 vs 1 scenario?

In [ ]:
LEFT_AGENT = "../input/open-gfball-agents/gfball_pattern_v15_submission.py"
RIGHT_AGENT = "../input/open-gfball-agents/tunable_baseline_bot.py"
output = agent_run(LEFT_AGENT, RIGHT_AGENT, DEBUG, CONFIG)
viz = visualize(output)
HTML(viz.to_html5_video())

It looks like gfootball pattern agent can improve in its shooting direction

<a id="example-2"></a>
# Example 2 - Kaggle 11 vs 11 scenario

Now we want to run kaggle 11 vs 11 full scenario pool player. We limit to only 3 teams here and 2 rounds, and set episode_steps to only half of normal game at 1500 in order to speed up these runs for this kernel.

With my local setup I can kick off a full run with all my agents and multiple rounds overnight and all the results will be done in the morning.

In [ ]:
AGENTS_DIR = '../input/open-gfball-agents/' # where the agents submission.py files are located
EXPORT_DIR = 'pool_play_results/' # optional flag to store results of pool play to files
NUM_ROUNDS = 2 # number of time to repeat playing every agent in pool

AGENTS = [
    '../input/open-gfball-agents/best_open_rules_bot_1020_v2.py',
    '../input/open-gfball-agents/tunable_baseline_bot.py',
    '../input/open-gfball-agents/gfball_pattern_v15_submission.py',
]

EPISODE_STEPS = 1500 # lets do shorter than default 3000 to speed up runs

SCENARIO_NAME = '11_vs_11_kaggle'

**This can take 5-10 minutes to run**

In [ ]:
%%time
result_df2, score_df2 = run_pool_play(SCENARIO_NAME, EPISODE_STEPS, NUM_ROUNDS, AGENTS,
                                    EXPORT_DIR, write_file=False)

The score dataframe has the results of running each agent against each other twice, (including against itself).

the number of points is based on 3 points for a win, 1 point for tie, 0 for loss

In [ ]:
score_df2

In [ ]:
print(result_df2.shape) # 3 agents in pool each play each other including self, (3x3) = 9 x 2 rounds = 18 matches
result_df2.head(10)

<a id="example-3"></a>
# Example 3 - Writing to files flag

In the case you want to store your different run results to file to keep track of, you can just set the write_file flag to True

In [ ]:
EXPORT_DIR = 'pool_play_results/' # optional flag to store results of pool play to files
NUM_ROUNDS = 3 # number of time to repeat playing every agent in pool

AGENTS = [
    '../input/open-gfball-agents/best_open_rules_bot_1020_v2.py',
    '../input/open-gfball-agents/tunable_baseline_bot.py',
    '../input/open-gfball-agents/gfball_pattern_v15_submission.py',
    'template_bot_submission.py'
]

EPISODE_STEPS = 400 # lets do shorter than default 3000 to speed up runs

SCENARIO_NAME = 'academy_3_vs_1_with_keeper'

In [ ]:
result_df, score_df = run_pool_play(SCENARIO_NAME, EPISODE_STEPS, NUM_ROUNDS, AGENTS,
                                    EXPORT_DIR, write_file=True)

Results are written to a dynamically created directory based on timestamp of run

In [ ]:
tdir = os.listdir('pool_play_results/')[0]
tdir

In [ ]:
os.listdir('pool_play_results/' + tdir)

A config file is written in the directory which has the configs that were run with

In [ ]:
config_df = pd.read_csv('pool_play_results/' + tdir + '/config.csv')
config_df

the results are stored in a results.csv file in the directory

In [ ]:
test_df = pd.read_csv('pool_play_results/' + tdir + '/results.csv')
test_df.head()

You can generate a scorecard from a results.csv file

In [ ]:
score_df = get_scoreboard_from_file('pool_play_results/' + tdir + '/results.csv')
score_df

Or you can generate a scorecard from a result dataframe

In [ ]:
score_df = get_scoreboard(test_df)
score_df